In [1]:
import urllib2
import numpy as np
import json
import pandas as pd
pd.set_option("display.max_columns",999)
from pandas.io.json import json_normalize
import os
import pickle

In [2]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext()

In [3]:
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)

In [4]:
#number per page = 30 
total_records = 1000
per_page = 30
n_pages=int(total_records/per_page)
allfiles ={}    
#loop to get all pages (n) (API only allows a max of 100 enteries per page)
for i in xrange(0, n_pages):
    try:
        print "Downloading Page {}".format(i)
        api ='http://api.inaturalist.org/v1/observations?page={}&quality_grade=any&identifications=any&captive=false&iconic_taxa%5B%5D=Protozoa&iconic_taxa%5B%5D=Animalia&iconic_taxa%5B%5D=Mollusca&iconic_taxa%5B%5D=Actinopterygii&iconic_taxa%5B%5D=Amphibia&iconic_taxa%5B%5D=Aves&iconic_taxa%5B%5D=Mammalia&iconic_taxa%5B%5D=Reptilia&iconic_taxa%5B%5D=Arachnida&iconic_taxa%5B%5D=Insecta&iconic_taxa%5B%5D=Plantae&iconic_taxa%5B%5D=Fungi&iconic_taxa%5B%5D=Chromista&iconic_taxa%5B%5D=unknown&place_id=14'.format(i)
        req = urllib2.Request(api)
        response = urllib2.urlopen(req)
        response_read = response.read()
        json_data=json.loads(response_read)
        allfiles[i] = json_data
    except:
        pass  

In [5]:
with open('iNat.json', 'w') as outfile:
    json.dump(allfiles, outfile)

In [6]:
#sparkdf = sqlsc.createDataFrame(iNat_df_main)
data = sqlsc.read.json('iNat.json')

In [7]:
data.printSchema()

root
 |-- 1: struct (nullable = true)
 |    |-- page: long (nullable = true)
 |    |-- per_page: long (nullable = true)
 |    |-- results: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- annotations: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- cached_votes_total: long (nullable = true)
 |    |    |    |-- captive: boolean (nullable = true)
 |    |    |    |-- comments: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- body: string (nullable = true)
 |    |    |    |    |    |-- created_at: string (nullable = true)
 |    |    |    |    |    |-- created_at_details: struct (nullable = true)
 |    |    |    |    |    |    |-- date: string (nullable = true)
 |    |    |    |    |    |    |-- day: long (nullable = true)
 |    |    |    |    |    |    |-- hour: long (nullable = true)
 |    |    |    |    |    |    |-

In [117]:
from pyspark.sql import functions as F
for i in range(1,n_pages):
    t = data.select("{}.results.captive".format(i), 
                "{}.results.cached_votes_total".format(i),
                '{}.results.comments_count'.format(i),
                '{}.results.community_taxon_id'.format(i),
                '{}.results.observed_on_details.day'.format(i),
                '{}.results.observed_on_details.hour'.format(i),
                '{}.results.observed_on_details.month'.format(i),
                '{}.results.observed_on_details.week'.format(i),
                '{}.results.observed_on_details.year'.format(i),
                '{}.results.faves_count'.format(i),
                '{}.results.geojson.coordinates'.format(i),
                '{}.results.geojson.type'.format(i),
                #'{}.results.geoprivacy'.format(i),
                '{}.results.id'.format(i),
                '{}.results.id_please'.format(i),
                '{}.results.identifications_count'.format(i),
                '{}.results.identifications_most_agree'.format(i),
                '{}.results.identifications_most_disagree'.format(i),
                '{}.results.identifications_some_agree'.format(i),
                #'{}.results.location'.format(i),
                '{}.results.mappable'.format(i),
                #'{}.results.obscured'.format(i),
                '{}.results.owners_identification_from_vision'.format(i),
                '{}.results.place_ids'.format(i),
                '{}.results.quality_grade'.format(i),
                #'{}.results.species_guess'.format(i),
                '{}.results.taxon.ancestor_ids'.format(i),
                '{}.results.taxon.complete_rank'.format(i),
                #'{}.results.taxon.introduced'.format(i),
                '{}.results.taxon.extinct'.format(i),
                '{}.results.taxon.iconic_taxon_name'.format(i),
                '{}.results.taxon.min_species_ancestry'.format(i),
                '{}.results.taxon.observations_count'.format(i),
                '{}.results.taxon.rank'.format(i),
                '{}.results.taxon.extinct'.format(i),
                '{}.results.taxon.threatened'.format(i),
                '{}.results.user.activity_count'.format(i),
                '{}.results.user.identifications_count'.format(i)
                   )
    if i == 1:
        alldata = t
    else:
        alldata = alldata.unionAll(t)

In [118]:
alldata.columns

['captive',
 'cached_votes_total',
 'comments_count',
 'community_taxon_id',
 'day',
 'hour',
 'month',
 'week',
 'year',
 'faves_count',
 'coordinates',
 'type',
 'id',
 'id_please',
 'identifications_count',
 'identifications_most_agree',
 'identifications_most_disagree',
 'identifications_some_agree',
 'mappable',
 'owners_identification_from_vision',
 'place_ids',
 'quality_grade',
 'ancestor_ids',
 'complete_rank',
 'extinct',
 'iconic_taxon_name',
 'min_species_ancestry',
 'observations_count',
 'rank',
 'extinct',
 'threatened',
 'activity_count',
 'identifications_count',
 'observations_count']

In [119]:
alldata_flat = alldata.rdd.flatMap(lambda row: [(captive,
                                                 cached_votes_total,
                                                 comments_count,
                                                 community_taxon_id,
                                                 day,
                                                 hour,
                                                 month,
                                                 week,
                                                 year,
                                                 faves_count,
                                                 coordinates,
                                                 type,
                                                 id,
                                                 id_please,
                                                 identifications_count,
                                                 identifications_most_agree,
                                                 identifications_most_disagree,
                                                 identifications_some_agree,
                                                 mappable,
                                                 owners_identification_from_vision,
                                                 place_ids,
                                                 quality_grade,
                                                 ancestor_ids,
                                                 complete_rank,
                                                 extinct,
                                                 iconic_taxon_name,
                                                 min_species_ancestry,
                                                 observations_count,
                                                 rank,
                                                 extinct,
                                                 threatened,
                                                 activity_count,
                                                 identifications_count
                                                ) 
                                   for captive,
                                                cached_votes_total,
                                                comments_count,
                                                community_taxon_id,
                                                day,
                                                hour,
                                                month,
                                                week,
                                                year,
                                                faves_count,
                                                coordinates,
                                                type,
                                                id,
                                                id_please,
                                                identifications_count,
                                                identifications_most_agree,
                                                identifications_most_disagree,
                                                identifications_some_agree,
                                                mappable,
                                                owners_identification_from_vision,
                                                place_ids,
                                                quality_grade,
                                                ancestor_ids,
                                                complete_rank,
                                                extinct,
                                                iconic_taxon_name,
                                                min_species_ancestry,
                                                observations_count,
                                                rank,
                                                extinct,
                                                threatened,
                                                activity_count,
                                                identifications_count
                                                 
                                   in zip(row.captive,
                                          row.cached_votes_total,
                                          row.comments_count,
                                          row.community_taxon_id,
                                          row.day,
                                          row.hour,
                                          row.month,
                                          row.week,
                                          row.year,
                                          row.faves_count,
                                          row.coordinates,
                                          row.type,
                                          row.id,
                                          row.id_please,
                                          row.identifications_count,
                                          row.identifications_most_agree,
                                          row.identifications_most_disagree,
                                          row.identifications_some_agree,
                                          row.mappable,
                                          row.owners_identification_from_vision,
                                          row.place_ids,
                                          row.quality_grade,
                                          row.ancestor_ids,
                                          row.complete_rank,
                                          row.extinct,
                                          row.iconic_taxon_name,
                                          row.min_species_ancestry,
                                          row.observations_count,
                                          row.rank,
                                          row.extinct,
                                          row.threatened,
                                          row.activity_count,
                                          row.identifications_count
                                                 )
                                               ]).toDF(['captive',
                                                        'cached_votes_total',
                                                        'comments_count',
                                                        'community_taxon_id',
                                                        'day',
                                                        'hour',
                                                        'month',
                                                        'week',
                                                        'year',
                                                        'faves_count',
                                                        'coordinates',
                                                        'type',
                                                        'id',
                                                        'id_please',
                                                        'identifications_count',
                                                        'identifications_most_agree',
                                                        'identifications_most_disagree',
                                                        'identifications_some_agree',
                                                        'mappable',
                                                        'owners_identification_from_vision',
                                                        'place_ids',
                                                        'quality_grade',
                                                        'ancestor_ids',
                                                        'complete_rank',
                                                        'extinct',
                                                        'iconic_taxon_name',
                                                        'min_species_ancestry',
                                                        'observations_count',
                                                        'rank',
                                                        'extinct',
                                                        'threatened',
                                                        'activity_count',
                                                        'identifications_count'
                                                       ])


In [120]:
alldata_flat.first()

Row(captive=False, cached_votes_total=0, comments_count=0, community_taxon_id=None, day=5, hour=14, month=1, week=1, year=2018, faves_count=0, coordinates=[u'-118.0825339113', u'33.8384769981'], type=u'Point', id=9377702, id_please=False, identifications_count=0, identifications_most_agree=False, identifications_most_disagree=False, identifications_some_agree=False, mappable=True, owners_identification_from_vision=False, place_ids=[1, 14, 962, 9853, 51043, 51727, 53512, 55105, 56773, 59613, 62068, 62332, 63047, 65360, 66068, 66741, 75903, 76101, 79780, 81264, 82017, 83134, 92151, 92337, 95699, 96034, 96343, 96683, 96687, 97394, 108490, 111374, 118252, 118253, 118386, 119138], quality_grade=u'needs_id', ancestor_ids=[48460, 1, 47120, 245097, 47119, 47118, 120474, 342614, 367200, 48139, 61881, 264380], complete_rank=None, extinct=False, iconic_taxon_name=u'Arachnida', min_species_ancestry=u'48460,1,47120,245097,47119,47118,120474,342614,367200,48139,61881,264380', observations_count=47, 